In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import re
import json


In [14]:
url = "https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne-a-vydajne-zdravotnickych-pomocok.html?page_id=3299&search_kraj=&iframe="
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
data = []
table = soup.find('table', attrs={'class':'list'})
rows = table.find_all('tr')
for row in rows:
    okres_link = row.find("a")['href']
    okres_nazov =  row.find("a").text.strip()
    page = requests.get(okres_link)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find('table', attrs={'class':'list'})
    rowsOkres = table.find_all('tr')
    for rows2 in rowsOkres:
        title_html = rows2.find('a')
        specialization = rows2.find('span').text.replace("(", " ").replace(")", "")
            
        title = title_html.text.strip()

        doctor_url = ""
        if title_html.has_attr('href'):
            doctor_url = title_html['href']
        

    


        data_row = {
            "title" : title,
            "doctor_url" : doctor_url,
            "doctor_id" : specialization,
            "kraj_id" : "7",
            "okres" : okres_nazov.split(" ")[1],

        }
        data.append(data_row)
df2_lekarne = pd.DataFrame(data)


In [15]:
df2_lekarne

,title,doctor_url,doctor_id,kraj_id,okres
0,Lekáreň CENTA,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-53467744-A0001,7,Bytča
1,"Výdajňa zdravotníckych pomôcok Medisamel, s.r.o.",https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-51949113-A0001,7,Bytča
2,TÍLIA,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-50735047-A0001,7,Bytča
3,Lekáreň Dr. Max,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-47781106-A0001,7,Bytča
4,Lekáreň Poliklinika Bytča,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-44222840-A0001,7,Bytča
...,...,...,...,...,...
272,Lekáreň FERULA,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-36435660-A0001,7,Žilina
273,EKOLEKÁREŇ,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-36365939-A0001,7,Žilina
274,Lekáreň Dr. Max,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-36778176-A0001,7,Žilina
275,Lekáreň Dr. Max,https://www.e-vuc.sk/zsk/zdravotnictvo/lekarne...,65-36731609-A0001,7,Žilina


In [18]:
lekaren_info = []
for index, lekaren in df2_lekarne.iterrows():
    url = lekaren['doctor_url']
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    table = soup.find('table', attrs={'class': 'data_table'})
    regex = "[^\w ]"
    rows = table.find_all('tr')

    info_dict = {}
    doctor_id = ""

    for row in rows:
        if row.find('th') != None:
            try:
                th = row.find('th').text.strip()
                th = re.sub(regex, "", th, count=0, flags=0).strip()

                td = row.find('td').text
                td_rows = []
                for td_row in td.split("\n"):
                    r = td_row.strip()
                    if r != "":
                        td_rows.append(r)

                info_dict[th] = "\n".join(td_rows)
                if th == "Identifikátor":
                    doctor_id = td_rows[0]
            except:
                pass

    tables_hours = soup.find_all(
        'table', attrs={"class": 'data_table no-full-width'})
    table_hours_text = []
    for hour_table in tables_hours:
        for tr in hour_table.find_all('tr'):
            if tr.find('th') != None:
                th = tr.find('th').text.strip()
            if tr.find('td') != None:
                td = tr.find('td').text.strip()
            table_hours_text.append(re.sub(r"\s ", "", th) + " " + re.sub(r"\s ", "", td))

    table_hours_text = "\n".join(table_hours_text)

    map = soup.find('a', attrs={'title': 'Mapa'})

    lat, lng = map['href'].split(":")[2].split(",")[0], map['href'].split(":")[2].split(",")[1]
    df_row = {
        "doctor_id" : doctor_id,
        "info" :  info_dict,
        "open_hours" : table_hours_text,
        "lat" : lat,
        "lng" : lng
    }
    lekaren_info.append(df_row)

In [37]:
lekaren_info_df = pd.DataFrame(lekaren_info)
lekaren_info_df['adresa'] = lekaren_info_df['info'].apply(lambda row :  row['Miesto prevádzkovania'])
lekaren_df = lekaren_info_df.join(df2_lekarne.drop(columns=["doctor_id"]))
lekaren_df['okres_id'] = lekaren_df['okres'].apply(lambda x : j_dic[x])

In [39]:
lekaren_df.to_csv("žilina_lekarne.csv", index=False)

In [36]:
# Opening JSON file
f = open('data/okres_ids.json')
  
# returns JSON object as 
# a dictionary
j = json.load(f)
j_dic = { x['name'] : x['id'] for x in j }
j_dic['Dolný'] = 12
j_dic['Kysucké'] = 27
j_dic['Liptovský'] = 30
j_dic['Turčianske'] = 68